# Example 4

- The k-out-of-n system
    - The system consists of $n$ **non-identical** compoments.
    - The system fails when $k$ compoments are down.
- Repairable system
    - A failed compoment may be repaired.
    - The repaired component is as good as new.
- Repairperson
    - Failed components are repaired by a single repairperson.
    - A failed component makes a queue with **a random** discipline
- The failure time of **i-th component** follows an exponential distribution with mean $1/\lambda_i$.
- The repair time for a failed component follows **a deterministic time**.

## Initialize

Load packages

In [ ]:
using Origin
using SparseMatrix
using NMarkov
using JuliaDot
using SparseArrays
using Plots
using MAT
using JSON
using Distributions
using Random

In [ ]:
# utility functions

function drawfile(x)
    data = open(x) do f
        read(f, String)
    end
    draw(data) # JuliaDot.draw
end

import NMarkov.eye
function eye(M::AbstractMatrix)
    eye(size(M)[1])
end

function speye(M::AbstractMatrix)
    n = size(M)[1]
    x = [i for i = 1:n]
    v = [1.0 for i = 1:n]
    sparse(x, x, v)
end

## Build the model with SPN

1. Make the SPN definition file (reliab3.spn)
1. Draw a diagram of SPN
1. Check the model behavior with one simulation

### Draw a petrinet diagram

In [ ]:
# generate a dot file to draw PN with gospn
run(`./gospn view -i reliab3.spn -o tmp.dot`)
# draw a picture with the dot file
drawfile("tmp.dot")

### Check the model behavior

In [ ]:
# set model params
rng = MersenneTwister(1234)
lambda = rand(rng, Uniform(0.01, 0.02), 10)
mu = 1.0
paramstring = join(["lambda$i = $(lambda[i]); " for i = 1:10]) * "mu = $mu;"

In [ ]:
# test the SPN model to check 30 transitions from the initial marking
run(`./gospn test -i reliab3.spn -n 30 -p "$paramstring"`)

## Construct CTMC matrices

1. Analyze SPN and generate the following files
    - A Matlab matrix file to store matrices (option: -o)
    - A dot file to draw the marking graph (reachability graph). In the case where SPN generates a large number of states, it is better not to generate this file (option: -m)
    - A dot file to draw the transitions between marking groups (option: -g)
2. Construct CTMC matrices from the matrix file and a diagrm of marking group

### Generate the marking graph

In [ ]:
# generate CTMC matricies, dot files for marking and group
run(`./gospn mark -i reliab3.spn -o result.mat -g gmark.dot -p "$paramstring"`)
# read CTMC matrices
matfile = matopen("result.mat") 
# draw a picture of groups of markings
drawfile("gmark.dot")

In [ ]:
# Read matrices
G0G0E = read(matfile, "G0G0E")
G0G1E = read(matfile, "G0G1E")
G1G1E = read(matfile, "G1G1E")
G1I0P0 = read(matfile, "G1I0P0")
G1I1P0 = read(matfile, "G1I1P0")
I0G0I = read(matfile, "I0G0I")
I1G1I = read(matfile, "I1G1I");

## Compute reliability measures
- Steady-state analysis for Markov Regenerative Process (MRGP)
    1. Construct a probability transitionn matrix of the embedded Markov chain (EMC) at instance when exiting form any groups
    2. Compute the steady-state probability vector of the EMC
    3. Compute the expected sojourn time for one transition of EMC
    4. Compute the steady-state probability of MRGP from the time franction of expected sojourn time
    5. Compute the steady-state reward by multiplying the steady-state probability vector with a reward vector

In [ ]:
# Make EMC

## Matrix on time instant of the end of state
V0 = -G0G0E \ eye(G0G0E)
V1, V1c = mexpc(G1G1E, eye(G1G1E), 1/mu) # constant distribution

# indicies when all states are concatinated [V0, V1]
tmp = [size(x)[1] for x = [V0, V1]]
start = 1
indices = []
for x = tmp
    push!(indices, start:start+x-1)
    start += x
end

## Transition probability matrices for EMC.
## This is constucted by the groupmark graph
P = spzeros(AbstractMatrix{Float64}, 2, 2) # blockmatrix
@origin P=>0 begin
    P[0,1] = V0 * G0G1E
    P[1,0] = V1 * G1I0P0 * I0G0I
    P[1,1] = V1 * G1I1P0 * I1G1I
end
P=sparse(block(P));

In [ ]:
# stationary vector for EMC
pid = gth(Matrix(P));

In [ ]:
# sojourn time & stationary distribution
S = spzeros(AbstractMatrix{Float64}, 2, 2) # blockmatrix
@origin S=>0 begin
    S[0,0] = V0
    S[1,1] = V1c
end
S=sparse(block(S))
sojourn = S' * pid
pis = sojourn / sum(sojourn);

In [ ]:
# reward vector
r0 = read(matfile, "availG0")
r1 = read(matfile, "availG1");

In [ ]:
@origin indices=>0 begin
    savail = sum(pis[indices[0]] .* r0) + sum(pis[indices[1]] .* r1)
end
savail # system availability